<a href="https://colab.research.google.com/github/dhayanesh/bert-embeddingExtraction/blob/main/bert_embeddingsExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=1e-5)

#reusing code for extracting updated indices from DLRM implementation
def get_nonzero_grad_indices(embedding_layer):
    nonzero_grad_indices = []

    def hook(grad):
        if grad.is_sparse:
            grad = grad.to_dense()
        unique_nonzero_indices = torch.unique(torch.nonzero(grad, as_tuple=False)[:, 0])
        nonzero_grad_indices.extend(unique_nonzero_indices.tolist())

    handle = embedding_layer.weight.register_hook(hook)
    return nonzero_grad_indices, handle

embedding_layer = model.bert.embeddings.word_embeddings
nonzero_grad_indices, handle = get_nonzero_grad_indices(embedding_layer)

for i in range(10):
    input_text = "Bidirectional Encoder Representations from Transformers (BERT) is a family of language models introduced by researchers at Google. BERT is based on the transformer architecture. Specifically, BERT is composed of Transformer encoder layers"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    labels = torch.tensor([1]).unsqueeze(0)
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

updated_embedding_values = {}
with torch.no_grad():
    for index in nonzero_grad_indices:
        updated_embedding = embedding_layer.weight.data[index].tolist()
        key = 'bert.embeddings.word_embeddings.weight'
        if key not in updated_embedding_values:
            updated_embedding_values[key] = {}
        updated_embedding_values[key][index] = updated_embedding

print(updated_embedding_values)
handle.remove()
